**NOTE**
The Trackman data is confidential, so the data used in this script is only to test 

In [ ]:


from pathlib import Path

from numpy import log10
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from custom_transforms import LoadSpectrogram, NormalizeSpectrogram, ToTensor, InterpolateSpectrogram
from data_management import make_dataset_name
from models import weights_init_uniform_rule, SpectrVelCNNRegr

MODEL = SpectrVelCNNRegr
LEARNING_RATE = 10**-4
WEIGHT_DECAY = 1e-4
EPOCHS = 350 # the model converges in test perfermance after ~250-300 epochs
BATCH_SIZE = 32
NUM_WORKERS = 0
OPTIMIZER = torch.optim.Adam
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Paths
DATA_ROOT = Path("data")
STMF_FILENAME = "stmf_data_3.csv"
train_dir = DATA_ROOT / "train"
test_dir = DATA_ROOT / "test"
validation_dir = DATA_ROOT / "validation"

# Transforms
TRAIN_TRANSFORM = Compose([
    LoadSpectrogram(root_dir=train_dir),
    # NormalizeSpectrogram(),
    ToTensor(),
    # InterpolateSpectrogram(size=(74, 918))
])

TEST_TRANSFORM = Compose([
    LoadSpectrogram(root_dir=test_dir),
    # NormalizeSpectrogram(),
    ToTensor(),
    # InterpolateSpectrogram(size=(74, 918))
])

VALIDATION_TRANSFORM = Compose([
    LoadSpectrogram(root_dir=validation_dir),
    # NormalizeSpectrogram(),
    ToTensor(),
    # InterpolateSpectrogram(size=(74, 918))
])

train_dataset = MODEL.dataset(data_dir= DATA_ROOT / "train",
                        stmf_data_path = DATA_ROOT / STMF_FILENAME,
                        transform=TRAIN_TRANSFORM)

validation_dataset = MODEL.dataset(data_dir= DATA_ROOT / "validation",
                        stmf_data_path = DATA_ROOT / STMF_FILENAME,
                        transform=TEST_TRANSFORM)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Test the dataloaders
for batch in train_dataloader:
    print(batch["spectrogram"].shape)
    print(batch["target"].shape)
    break


: 